In [4]:
import pandas as pd
import numpy as np
import json
import requests
from bs4 import BeautifulSoup
import folium
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import re


### List of stations and geographical coordinates


Data downloaded from wikipedia
https://en.wikipedia.org/wiki/List_of_Chicago_%22L%22_stations

In [92]:
stations=pd.read_csv("List_of_Station_Chicago.csv")
df.head()

,Station,Lines,Transfers,Location,Opened,Structure
0,O'Hare,Blue,O'Hare: ATS Airport Transit System,O'Hare,"September 3, 1984[12]",Underground
1,Rosemont,Blue,—,Rosemont,"February 27, 1983[13]",Expressway median
2,Cumberland,Blue,—,O'Hare,"February 27, 1983[13]",Expressway median
3,Harlem,Blue,—,Norwood Park,"February 27, 1983[13]",Expressway median
4,Jefferson Park,Blue,Metra: UP-NW,Jefferson Park,"February 1, 1970[14]",Expressway median


In [89]:
df.Lines.value_counts()

Blue                              33
Red                               29
Green                             22
Pink                              11
Brown                             11
Purple                             8
Orange                             7
Brown Purple                       6
Brown Orange Pink Purple           4
Brown Green Orange Pink Purple     4
Green Pink                         3
Yellow                             2
Brown Purple Red                   2
Red Purple                         1
Green Orange                       1
Red Purple  Yellow                 1
Name: Lines, dtype: int64

In [133]:
station_dict={}
import time
start = time.time()

for i in stations.Location:
    address = i + ', ' + 'Chicago, Illinois,United States'
    location = geocoder.geocode(address)
    latlng = location[0]['geometry']
    station_dict[i]=latlng
    print(i,latlng,str(time.time()-start)+'s')
station_dict

O'Hare {'lat': 41.973101, 'lng': -87.9067679} 1.8502628803253174s
Rosemont {'lat': 41.9468532, 'lng': -87.7947805} 2.7468299865722656s
O'Hare {'lat': 41.973101, 'lng': -87.9067679} 3.585963010787964s
Norwood Park {'lat': 41.9855895, 'lng': -87.8005772} 4.362934112548828s
Jefferson Park {'lat': 41.9697375, 'lng': -87.7631179} 5.169133901596069s
Irving Park {'lat': 41.953365, 'lng': -87.7364471} 5.989357948303223s
Irving Park {'lat': 41.953365, 'lng': -87.7364471} 6.8168559074401855s
Avondale / Irving Park {'lat': 41.953365, 'lng': -87.7364471} 7.695746898651123s
Avondale {'lat': 41.9389208, 'lng': -87.711168} 8.42103099822998s
Logan Square {'lat': 41.925221, 'lng': -87.7013096} 9.183284997940063s
Logan Square {'lat': 41.925221, 'lng': -87.7013096} 9.975032806396484s
Logan Square {'lat': 41.925221, 'lng': -87.7013096} 10.741435050964355s
Wicker Park {'lat': 41.9078018, 'lng': -87.6781551} 11.62657380104065s
Wicker Park / Noble Square {'lat': 41.905809, 'lng': -87.672125} 12.3951599597930

Rogers Park {'lat': 42.0105314, 'lng': -87.6707482} 91.8647928237915s
Edgewater {'lat': 41.9833689, 'lng': -87.6639516} 92.85334587097168s
Edgewater {'lat': 41.9833689, 'lng': -87.6639516} 93.55515813827515s
Edgewater {'lat': 41.9833689, 'lng': -87.6639516} 94.26667499542236s
Edgewater {'lat': 41.9833689, 'lng': -87.6639516} 94.95652198791504s
Little Vietnam {'lat': 41.9732234, 'lng': -87.6587651} 95.68473792076111s
Uptown {'lat': 41.9666299, 'lng': -87.6555458} 96.44716095924377s
Uptown {'lat': 41.9666299, 'lng': -87.6555458} 97.1435329914093s
Lake View {'lat': 41.94705, 'lng': -87.6554288} 97.8467788696289s
Wrigleyville {'lat': 41.947022, 'lng': -87.6564773} 98.52696204185486s
Near North Side {'lat': 41.9000327, 'lng': -87.6344975} 99.28257489204407s
Near North Side {'lat': 41.9000327, 'lng': -87.6344975} 100.20050692558289s
Near North Side {'lat': 41.9000327, 'lng': -87.6344975} 100.89918804168701s
River North / Streeterville {'lat': 41.890808, 'lng': -87.611113} 101.7614438533783s


{"O'Hare": {'lat': 41.973101, 'lng': -87.9067679},
 'Rosemont': {'lat': 41.9468532, 'lng': -87.7947805},
 'Norwood Park': {'lat': 41.9855895, 'lng': -87.8005772},
 'Jefferson Park': {'lat': 41.9697375, 'lng': -87.7631179},
 'Irving Park': {'lat': 41.953365, 'lng': -87.7364471},
 'Avondale / Irving Park': {'lat': 41.953365, 'lng': -87.7364471},
 'Avondale': {'lat': 41.9389208, 'lng': -87.711168},
 'Logan Square': {'lat': 41.925221, 'lng': -87.7013096},
 'Wicker Park': {'lat': 41.9078018, 'lng': -87.6781551},
 'Wicker Park / Noble Square': {'lat': 41.905809, 'lng': -87.672125},
 'West Town': {'lat': 41.9014213, 'lng': -87.6861661},
 'The Loop': {'lat': 41.8788844, 'lng': -87.6271806},
 'Near West Side': {'lat': 41.8800656, 'lng': -87.6667624},
 'Greektown': {'lat': 41.8784447, 'lng': -87.6469318},
 'East Garfield Park': {'lat': 41.8808657, 'lng': -87.7028328},
 'West Garfield Park': {'lat': 41.8805878, 'lng': -87.7292227},
 'Austin': {'lat': 41.8878759, 'lng': -87.7648508},
 'Oak Park': 

In [134]:
latlng=pd.DataFrame.from_dict(station_dict,orient='index').reset_index()
latlng.columns=['Station','latitude','longitude']
station_new=pd.merge(stations,latlng,how='left', on=['Station'])
station_new

,Station,Lines,Transfers,Location,Opened,Structure,latitude,longitude
0,O'Hare,Blue,O'Hare: ATS Airport Transit System,O'Hare,"September 3, 1984[12]",Underground,41.973101,-87.906768
1,Rosemont,Blue,—,Rosemont,"February 27, 1983[13]",Expressway median,41.946853,-87.794781
2,Cumberland,Blue,—,O'Hare,"February 27, 1983[13]",Expressway median,NaN,NaN
3,Harlem,Blue,—,Norwood Park,"February 27, 1983[13]",Expressway median,NaN,NaN
4,Jefferson Park,Blue,Metra: UP-NW,Jefferson Park,"February 1, 1970[14]",Expressway median,41.969738,-87.763118
5,Montrose,Blue,Metra: MD-N (at Mayfair),Irving Park,"February 1, 1970[14]",Expressway median,NaN,NaN
6,Irving Park,Blue,Metra: UP-NW (at Irving Park),Irving Park,"February 1, 1970[14]",Expressway median,41.953365,-87.736447
7,Addison,Blue,—,Avondale / Irving Park,"February 1, 1970[14]",Expressway median,NaN,NaN
8,Belmont,Blue,—,Avondale,"February 1, 1970[14]",Underground,NaN,NaN
9,Logan Square,Blue,—,Logan Square,"May 25, 1895[15]",Underground,41.925221,-87.701310


Neighbourhood df download from wiki
https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Chicago

In [96]:
neighbours = pd.read_csv('List_of_neighborhoods_in_Chicago.csv')
neighbours.head()

,Neighborhood,Community area
0,Albany Park,Albany Park
1,Altgeld Gardens,Riverdale
2,Andersonville,Edgewater
3,Archer Heights,Archer Heights
4,Armour Square,Armour Square


In [ ]:
# # !pip3 install geopy
# from geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="Chicago_explorer",timeout=3)
# neighbours['Coord'] = neighbours['Neighborhood'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitute))
# neighbours

In [110]:
# !pip3 install opencage
from opencage.geocoder import OpenCageGeocode
key = '20b19d9ca52c4af8b4ee86a53b693aa6'
geocoder = OpenCageGeocode(key)

In [118]:
adict={}
import time
start = time.time()

for i in neighbours.Neighborhood:
    address = i + ', ' + 'Chicago, Illinois,United States'
    location = geocoder.geocode(address)
    latlng = location[0]['geometry']
    adict[i]=latlng
    print(i,latlng,str(time.time()-start)+'s')
adict

Albany Park {'lat': 41.9719367, 'lng': -87.7161739} 1.9347128868103027s
Altgeld Gardens {'lat': 41.6548637, 'lng': -87.6004462} 2.7714791297912598s
Andersonville {'lat': 41.9771393, 'lng': -87.6692734} 3.586030960083008s
Archer Heights {'lat': 41.8114215, 'lng': -87.7261651} 4.277545928955078s
Armour Square {'lat': 41.8400333, 'lng': -87.633107} 5.017735958099365s
Ashburn {'lat': 41.7475332, 'lng': -87.7111628} 5.903697967529297s
Ashburn Estates {'lat': 41.88531, 'lng': -87.62213} 6.653886079788208s
Auburn Gresham {'lat': 41.7433872, 'lng': -87.6560416} 7.411498069763184s
Avalon Park {'lat': 41.7450346, 'lng': -87.5886584} 8.161303997039795s
Avondale {'lat': 41.9389208, 'lng': -87.711168} 8.905597925186157s
Avondale Gardens {'lat': 41.9389208, 'lng': -87.711168} 9.69037914276123s
Back of the Yards {'lat': 41.8075332, 'lng': -87.6661628} 10.423412084579468s
Belmont Central {'lat': 41.9397964, 'lng': -87.6533276} 11.335841178894043s
Belmont Gardens {'lat': 41.9397964, 'lng': -87.6533276}

Lincoln Park {'lat': 41.9402976, 'lng': -87.6381171} 90.71778583526611s
Lincoln Square {'lat': 41.9759898, 'lng': -87.6896163} 91.44337487220764s
Lithuanian Plaza {'lat': 41.6869288, 'lng': -87.7306849} 92.16768288612366s
Little Italy {'lat': 41.873733, 'lng': -87.658413} 92.94821095466614s
Little Village {'lat': 41.861876, 'lng': -87.647786} 93.67852401733398s
Logan Square {'lat': 41.9285683, 'lng': -87.7067931} 94.47920393943787s
Longwood Manor {'lat': 41.6903981, 'lng': -87.6721901} 95.41637516021729s
The Loop {'lat': 41.8816094, 'lng': -87.6294574} 96.27477407455444s
Lower West Side {'lat': 41.8541996, 'lng': -87.6656086} 97.01648378372192s
Loyola {'lat': 42.0013525, 'lng': -87.6613062} 97.82752585411072s
Magnificent Mile {'lat': 41.8945229, 'lng': -87.624228} 98.55590891838074s
Margate Park {'lat': 41.972484, 'lng': -87.650708} 99.26766180992126s
Marquette Park {'lat': 41.7685506, 'lng': -87.7028826} 100.02799797058105s
Marshall Square {'lat': 41.8522013, 'lng': -87.6940542} 100.8

West Elsdon {'lat': 41.7939217, 'lng': -87.7244978} 181.16029381752014s
West Englewood {'lat': 41.7780891, 'lng': -87.6667177} 181.90803599357605s
West Garfield Park {'lat': 41.8805878, 'lng': -87.7292227} 182.65902185440063s
West Humboldt Park {'lat': 41.9057668, 'lng': -87.7041737} 183.57091522216797s
West Lakeview {'lat': 41.9439185, 'lng': -87.6540759} 184.32903599739075s
West Lawn {'lat': 41.7728107, 'lng': -87.722275} 185.06026697158813s
West Loop {'lat': 41.8823318, 'lng': -87.6385777} 186.0804159641266s
West Morgan Park {'lat': 41.6903117, 'lng': -87.6667156} 187.02170491218567s
West Pullman {'lat': 41.6750456, 'lng': -87.637823} 187.80446910858154s
West Ridge {'lat': 42.0035482, 'lng': -87.6962426} 188.5412209033966s
West Rogers Park {'lat': 42.0105314, 'lng': -87.6707482} 189.40524077415466s
West Town {'lat': 41.9014213, 'lng': -87.6861661} 190.14655685424805s
West Woodlawn {'lat': 41.7794786, 'lng': -87.599493} 191.0037682056427s
Wicker Park {'lat': 41.9078018, 'lng': -87.67

{'Albany Park': {'lat': 41.9719367, 'lng': -87.7161739},
 'Altgeld Gardens': {'lat': 41.6548637, 'lng': -87.6004462},
 'Andersonville': {'lat': 41.9771393, 'lng': -87.6692734},
 'Archer Heights': {'lat': 41.8114215, 'lng': -87.7261651},
 'Armour Square': {'lat': 41.8400333, 'lng': -87.633107},
 'Ashburn': {'lat': 41.7475332, 'lng': -87.7111628},
 'Ashburn Estates': {'lat': 41.88531, 'lng': -87.62213},
 'Auburn Gresham': {'lat': 41.7433872, 'lng': -87.6560416},
 'Avalon Park': {'lat': 41.7450346, 'lng': -87.5886584},
 'Avondale': {'lat': 41.9389208, 'lng': -87.711168},
 'Avondale Gardens': {'lat': 41.9389208, 'lng': -87.711168},
 'Back of the Yards': {'lat': 41.8075332, 'lng': -87.6661628},
 'Belmont Central': {'lat': 41.9397964, 'lng': -87.6533276},
 'Belmont Gardens': {'lat': 41.9397964, 'lng': -87.6533276},
 'Belmont Heights': {'lat': 41.940104, 'lng': -87.640367},
 'Belmont Terrace': {'lat': 41.9397964, 'lng': -87.6533276},
 'Beverly': {'lat': 41.7181532, 'lng': -87.6717674},
 'Beve

In [128]:
latlng=pd.DataFrame.from_dict(adict,orient='index').reset_index()
latlng.columns=['Neighborhood','latitude','longitude']

In [129]:
neighbours_new=pd.merge(neighbours,latlng,how='left', on=['Neighborhood'])

In [130]:
neighbours_new.head()

,Neighborhood,Community area,latitude,longitude
0,Albany Park,Albany Park,41.971937,-87.716174
1,Altgeld Gardens,Riverdale,41.654864,-87.600446
2,Andersonville,Edgewater,41.977139,-87.669273
3,Archer Heights,Archer Heights,41.811422,-87.726165
4,Armour Square,Armour Square,41.840033,-87.633107


In [131]:
neighbours_new.shape

(246, 4)

In [117]:
client_id="1QWBEEEJMMQ1TD2ZOOO31GHSEWRJ24N3Y40EHQR2ZEH0PI4A"
client_secret="TB15G114QVYLV0OGEN3Q4C2SWO0DF2VOYPU24WEKDQZRLGB1"

In [132]:
#create map of Chicago with all neighbourhood
# 41.8781° N, 87.6298° W
map_chicago_neig = folium.Map(location=[41.8781, -87.6298], zoom_start=10)
#add markers
for neighborhood, lat, lng in zip(neighbours_new['Neighborhood'], neighbours_new['latitude'],neighbours_new['longitude']):
    #Also add a 500-meter circle around the station to visualize our neighborhoods
    #folium.Circle(
    #    latlong,        
    #    radius=1000,
    #    color=line_colors[line]
    #).add_to(map_moscow_metro)
    #Add marker with popup
    folium.Circle(
        [lat,lng],
        popup=neighborhood,
        radius=20,
        color='red'
    ).add_to(map_chicago_neig)  
map_chicago_neig

In [ ]:
#create map of Chicago with all station
# 41.8781° N, 87.6298° W
map_chicago_station = folium.Map(location=[41.8781, -87.6298], zoom_start=10)
#add markers
for station, lat, lng in zip(stations['Station'], stations['latitude'],neighbours_new['longitude']):
    #Also add a 500-meter circle around the station to visualize our neighborhoods
    #folium.Circle(
    #    latlong,        
    #    radius=1000,
    #    color=line_colors[line]
    #).add_to(map_moscow_metro)
    #Add marker with popup
    folium.Circle(
        [lat,lng],
        popup=neighborhood,
        radius=20,
        color='red'
    ).add_to(map_chicago_neig)  
map_chicago_neig